# Linear interpolation of semi-structured data in three and five dimensions

I recently encountered a problem in which billions of points needed to be linearly interpolated onto a [regular grid](https://en.wikipedia.org/wiki/Regular_grid) in five-dimensional (5D) space. I found that the task is infeasible for a large number of points in an unstructured data set since the time-complexity of Delauny triangulation is O(...). In my case, the data is already on a grid in some dimensions, but the grids are tilted: calling the variables $\left\{x_1, x_2, x_3, x_4, x_5\right\}$, $x_1$-$x_2$ and $x_3$-$x_4$ are on tilted grids while $x_5$ is unstructured. I will call this data *semi-structured*. (Is there a better word for this?)

Such a data set can be interpolated onto a regular 5D grid without running a 5D interpolation:
1. Interpolate $x_5$ for each $\left\{x_1, x_2, x_3, x_4\right\}$ to get $\tilde{x}_5$, where the tilde represents that the coordinate is on a regular grid.
2. Interpolate $x_2$-$x_3$ for each $\left\{x_3, x_4, \tilde{x}_5\right\}$ to get $\tilde{x}_5$ and $\tilde{x}_5$.
3. Interpolate $x_3$-$x_4$ for each $\left\{\tilde{x}_1, \tilde{x}_2, \tilde{x}_5\right\}$ to get $\tilde{x}_3$ and $\tilde{x}_4$.

Although this seemed like the correct approach, I needed to check that it would work on artificially generated data.

## Background: Delauney triangulation 

The time-complexity of Delauney triangulation for $n$ points in $d$-dimensional space is something like $O(n \log n)$ when $d \le 3$. The situation is much worse when $d > 3$; it was been shown that the worst-scase time-complexity is $O(n^{d/2})$ [cite Change: A Polynomial Time Algorithm for Multivariate Interpolation in Arbitrary Dimension via the Delaunay Triangulation], and that typical cases still scale exponentially.

In [ ]:
import sys
import time
import numpy as np
from scipy import interpolate
from scipy import ndimage
from matplotlib import pyplot as plt
from matplotlib import animation
import proplot as pplt
from tqdm import tqdm
from tqdm import trange

import utils_2022_07_13 as utils

pplt.rc['animation.html'] = 'jshtml'
pplt.rc['cmap.discrete'] = False
pplt.rc['cmap.sequential'] = 'viridis'
pplt.rc['grid'] = False
pplt.rc['figure.facecolor'] = 'white'
pplt.rc['savefig.dpi'] = 'figure'

In [ ]:
#hide
state = np.random.RandomState()
state.seed(0)

## 3D testing 

First, I will test this method on a 3D dataset, for which only steps 1 and 2 are necessary.

In [ ]:
ndim = 3  # number of dimensions
n = 40  # grid points per dimension
shape = tuple(ndim * [n])
labels = [r'$x_{}$'.format(i + 1) for i in range(ndim)]

# Create "normalized" coordinate arrays
xmax = 4.0  
coords_n = [np.linspace(-xmax, xmax, shape[i]) for i in range(ndim)]
COORDS_n = np.meshgrid(*coords_n, indexing='ij')

# Tilt the x1-x2 grid.
tilt = 0.5
x1_n, x2_n, x3_n = coords_n
X1_n, X2_n, X3_n = COORDS_n
X1 = X1_n + tilt * X2_n
X2 = X2_n
X3 = X3_n

# Randomize the spacing of the x3 coordinates for each x1 and x2.
noise = 0.02
X3 = np.zeros(shape)
for i in range(shape[0]):
    for j in range(shape[1]):
        X3[i, j, :] = X3_n[i, j, :] * np.random.uniform(1.0 - noise, 1.0 + noise, shape[2])
        X3[i, j, :] = np.sort(X3[i, j, :])
COORDS = [X1, X2, X3]

In [ ]:
#hide_input
fig, axes = pplt.subplots(nrows=3, ncols=3, figwidth=5, spanx=False, spany=False)
for i in range(3):
    for j in range(3):
        axes[i, j].scatter(COORDS[j].ravel(), COORDS[i].ravel(), 
                           color='black', s=0.1, alpha=0.5)
    axes[i, 0].format(ylabel=dims[i])
    axes[-1, i].format(xlabel=dims[i])
axes.format(suptitle='2D views of the grid points', suptitle_kw=dict(fontweight='normal'))
plt.show()

Define a random function on the skewed grid.

In [ ]:
def func(X1, X2, X3, tilt=1.0, cut=None, sigma=1.0):
    R = np.sqrt(X1**2 + X2**2 + X3**2)
    A = np.abs(np.sin(2.0 * R))**2
    f = A * np.exp(-0.5 * (R**2 - tilt*X1*X2 + 0.2*X1*X3 + 2.0*X2*X3))
    if cut is not None:
        f[R > cut] = 0.0
    f = ndimage.gaussian_filter(f, sigma=sigma)
    f = f / np.sum(f)
    return f

f = func(X1, X2, X3, tilt=tilt, cut=xmax)

Here are some of the $x_1$-$x_2$ projections as $x_3$ is varied. 

In [ ]:
#collapse
kk = np.linspace(0, shape[2] - 1, 10).astype(int)
vmax = np.max(f[:, :, kk])
vmin = np.min(f[:, :, kk])

fig, axes = pplt.subplots(ncols=5, nrows=2, figwidth=7.0)
for k, ax in zip(kk, axes):
    ax.pcolormesh(X1[:, :, k].T, X2[:, :, k].T, f[:, :, k].T,
                  vmin=vmin, vmax=vmax, ec='None')
    ax.format(xlabel=labels[0], ylabel=labels[1])
    ax.set_title(r'$x_3$ = {:.2f}'.format(X3[0, 0, k]), fontsize='medium')
axes.format(xticks=[], yticks=[])

Define the a regular grid on which to interpolate. Since the grid is tilted, it makes sense to increase the resolution along the $x_1$ axis.

In [ ]:
new_shape = tuple(np.multiply(shape, [1.5, 1.0, 1.1]).astype(int))
new_coords = [np.linspace(COORDS[i].min(), COORDS[i].max(), new_shape[i]) 
              for i in range(ndim)]

Let's test the interpolation on a 2D slice of the 3D array. The red dots on the left plot show the points on the interpolation grid.

In [ ]:
def normalize(f):
    f = np.clip(f, 0.0, None)
    f = f / np.sum(f)
    return f

idx = (slice(None), slice(None), shape[2] // 2)
points = (X1[idx].ravel(), X2[idx].ravel())
values = f[idx].ravel()
new_points = tuple([C.ravel() for C in np.meshgrid(*new_coords[:2], indexing='ij')])
grid_kws = dict(method='linear', fill_value=0.0)
new_values = interpolate.griddata(points, values, new_points, **grid_kws)
f2d = new_values.reshape(new_shape[0], new_shape[1])
f2d = normalize(f2d)

fig, axes = pplt.subplots(ncols=2)
axes[0].pcolormesh(X1[idx].T, X2[idx].T, f[idx].T)
axes[1].pcolormesh(new_coords[0], new_coords[1], f2d.T)
for i in range(len(new_coords[0])):
    for j in range(len(new_coords[1])):
        axes[0].scatter(new_coords[0][i], new_coords[1][j], color='red', ec='None', s=0.5)
axes.format(xlabel=labels[0], ylabel=labels[1], toplabels=['Initial', 'Interpolated'])
plt.show()

### Method A: 3D interpolation

Direct 3D interpolation using `scipy.griddata` — fitting a function of three variables to the data — does not take too long in this case; this will be our benchmark.

In [ ]:
start_time = time.time()

points = tuple([C.ravel() for C in COORDS])
values = f.ravel()
new_points = tuple([C.ravel() for C in np.meshgrid(*new_coords, indexing='ij')])
new_values = interpolate.griddata(points, values, new_points, **grid_kws)
f_new_3d = normalize(new_values.reshape(new_shape))

print(f'{(time.time() - start_time):2f} sec')

### Method B: 1D + 2D interpolation

As described above, we don't need to perform the full 3D interpolation; we can take advantage of the original grid structure. First, interpolate over $x_3$ for each ($x_1$-$x_2$), storing the result in `_f`.

In [ ]:
start_time = time.time()

_f = np.copy(f)
f_new = np.zeros((shape[0], shape[1], new_shape[2]))
new_points = new_coords[2]
for i in range(shape[0]):
    for j in range(shape[1]):
        points = X3[i, j, :]
        values = _f[i, j, :]
        f_new[i, j, :] = interpolate.griddata(points, values, new_points, **grid_kws)
        
print(f'{(time.time() - start_time):2f} sec')

We now need to make new coordinate arrays. All that we need to do is copy the existing $x_1$ and $x_2$ grids into the third dimension, then copy the $\tilde{x}_3$ grid into the first and second dimensions.

In [ ]:
def copy_into_new_dim(array, shape, axis=-1, method='broadcast'):
    """Copy an array into one or more new dimensions.
        
    Parameters
    ----------
    array : ndarray
        The array to copy.
    shape : tuple
        The shape of the copied array in the new dimensions. Example: if `shape=(2, 3)`
        and `array.shape=(4, 5)`, the shape of the output array could be (2, 3, 4, 5). 
    axis : int
        Where to copy the array. If 0, the copies will go "behind" the array; if -1,
        the copies will go "in front" of the array. I haven't tried anything else. For
        example, if `shape=(2, 3)` and `array.shape=(4, 5)` and `axis=0`, the shape 
        of the output array will be (4, 5, 2, 3). If `axis=1`, the the shape of the
        output array will be (2, 3, 4, 5). 
    method : {'broadcast', 'repeat'}
        The 'broadcast' method is faster since it works with views instead of copies. 
        See 'https://stackoverflow.com/questions/32171917/how-to-copy-a-2d-array-into-a-3rd-dimension-n-times'
    copy : bool
        If `method='broadcast'`, whether to return a view or copy.
    """
    if type(shape) in [int, np.int32, np.int64]:
        shape = (shape,)
    if method == 'repeat':
        for i in range(len(shape)):
            array = np.repeat(np.expand_dims(array, axis), shape[i], axis=axis)
        return array
    elif method == 'broadcast':
        if axis == 0:
            new_shape = shape + array.shape
        elif axis == -1:
            new_shape = array.shape + shape
        for _ in range(len(shape)):
            array = np.expand_dims(array, axis)
        return np.broadcast_to(array, new_shape)
    else:
        raise ValueError('Invalid `method` parameter.')

In [ ]:
X1 = copy_into_new_dim(X1[:, :, 0], (new_shape[2],), axis=-1)
X2 = copy_into_new_dim(X2[:, :, 0], (new_shape[2],), axis=-1)
X3 = copy_into_new_dim(new_coords[2], (shape[0], shape[1],), axis=0)
COORDS = [X1, X2, X3]

print('New coordinate arrays:')
for i in range(ndim):
    print(f'X{i + 1}.shape = : {COORDS[i].shape}')

Now the second step: perform a 2D interpolation on the $x_1$-$x_2$ distribution at each $\tilde{x}_3$ slice.

In [ ]:
start_time = time.time()

_f = np.copy(f_new)
f_new = np.zeros(new_shape)
new_points = tuple([C.ravel() for C in np.meshgrid(*new_coords[:2], indexing='ij')])
for k in range(new_shape[2]):
    points = (X1[:, :, k].ravel(), X2[:, :, k].ravel())
    values = _f[:, :, k].ravel()
    new_values = interpolate.griddata(points, values, new_points, **grid_kws)
    f_new[:, :, k] = new_values.reshape(new_shape[0], new_shape[1])
f_new = normalize(f_new)    

print(f'{(time.time() - start_time):2f} sec')

And we're done! The last step is unnecessary but will be needed for the 5D data set: reshape the grids once again.

In [ ]:
_X1, _X2 = np.meshgrid(new_coords[0], new_coords[1], indexing='ij')
X1 = copy_into_new_dim(_X1, (new_shape[2],), axis=-1)
X2 = copy_into_new_dim(_X2, (new_shape[2],), axis=-1)
X3 = copy_into_new_dim(X3[0, 0, :], (new_shape[0], new_shape[1]), axis=0)
COORDS = [X1, X2, X3]

print('New coordinate arrays:')
for i in range(ndim):
    print(f'X{i + 1}.shape = : {COORDS[i].shape}')

### Comparison 

A good way to compare 3D distributions is to slice along one dimensions. We need to compare `f_new` (the 2D + 1D interpolation method), `f_new_3D` (the 3D interpolation method), and `f_true` (the function evaluated on the interpolation grid). In the following figure, the color map scale is the same for all frames. The 1D projections of the images are plotted as faint lines on the bottom and left side of each frame.

In [ ]:
f_true = func(*COORDS, tilt=tilt, cut=xmax)

In [ ]:
ncols = 7
kk = np.linspace(0, new_shape[2] - 1, ncols + 2)[1:-1].astype(int)
prof_kws = dict(lw=0.5, alpha=0.5, scale=0.1)

fig, axes = pplt.subplots(ncols=ncols, nrows=3, figwidth=7.0, space=0.5)
for row, array in enumerate([f_true, f_new_3d, f_new]):
    vmin = np.min(array[:, :, kk])
    vmax = np.max(array[:, :, kk])
    for ax, k in zip(axes[row, :], kk):
        idx = (slice(None), slice(None), k)
        utils.plot_image(array[idx], x=new_coords[0], y=new_coords[1], ax=ax,
                         profx=True, profy=True, prof_kws=prof_kws,
                         vmin=vmin, vmax=vmax)
        if row == 0:
            ax.set_title(f'{labels[2]} = {new_coords[2][k]:.2f}', fontsize=8.5)
axes.format(
    xticks=[], yticks=[], 
    xlabel=labels[0], ylabel=labels[1],
    leftlabels=['True', 'int. (3D)', 'int. (1D + 2D)'],
    leftlabels_kw=dict(fontsize='medium'),
)

It looks like both methods gave the same answer. We can also look at the full 2D projections of the array. (It is very helpful to use `ipywidgets` to interactively slice through the data, but I haven't figured out how to do something like this on this blog, without a live kernel.) 

In [ ]:
for arr, title in zip([f_true, f_new_3d, f_new], 
                      ['True', 'Interpolated (3D)', 'Interpolated (1D + 2D)']):
    axes = utils.corner(arr, coords=new_coords, diag_kind=None, labels=labels, 
                        prof=True, prof_kws=prof_kws,
                        fig_kws=dict(figwidth=3.5))
    axes.format(suptitle=title)
    plt.show()

## 5D testing

In the 5D case, we have an extra interpolation to perform. Here again is the method:
1. Interpolate $x_5$ for each $\left\{x_1, x_2, x_3, x_4\right\}$ to get $\tilde{x}_5$, where the tilde represents that the coordinate is on a regular grid.
2. Interpolate $x_2$-$x_3$ for each $\left\{x_3, x_4, \tilde{x}_5\right\}$ to get $\tilde{x}_5$ and $\tilde{x}_5$.
3. Interpolate $x_3$-$x_4$ for each $\left\{\tilde{x}_1, \tilde{x}_2, \tilde{x}_5\right\}$ to get $\tilde{x}_3$ and $\tilde{x}_4$.

The testing will proceed in the same way as the 3D case, but we will need to decrease the resolution. We also will not be able to test the direct 5D interpolation method due to the poor scaling of Delauney triangulation to higher dimensions. But we can still compare with the function evaluated on the interpolation grid.

In [ ]:
ndim = 5  # number of dimensions
n = 20  # grid points per dimension
shape = tuple(ndim * [n])
labels = [f'x{i + 1}' for i in range(ndim)]

# Create "normalized" coordinate arrays
coords_n = [np.linspace(-xmax, xmax, shape[i]) for i in range(ndim)]
COORDS_n = np.meshgrid(*coords_n, indexing='ij')

# Tilt the x1-x2 and x3-x4 grids.
tilt = 0.5
x1_n, x2_n, x3_n, x4_n, x5_n = coords_n
X1_n, X2_n, X3_n, X4_n, X5_n = COORDS_n
X1 = X1_n + tilt * X2_n
X2 = X2_n
X3 = X3_n - tilt * X4_n
X4 = X4_n

# Randomize the spacing of the x5 coordinates for each {x1, x2, x3, x3}.
noise = 0.02
X5 = np.zeros(shape)
for i in range(shape[0]):
    for j in range(shape[1]):
        for k in range(shape[2]):
            for l in range(shape[3]):
                idx = (i, j, k, l, slice(None))
                X5[idx] = X5_n[idx] * np.random.uniform(1.0 - noise, 1.0 + noise, shape[2])
                X5[idx] = np.sort(X5[idx])
COORDS = [X1, X2, X3, X4, X5]

Define a 5D function.

In [ ]:
def func(X1, X2, X3, X4, X5, tilt=1.0, cut=None, sigma=1.0):
    R = np.sqrt(X1**2 + X2**2 + X3**2 + X4**2 + X5**2)
    A = np.abs(np.sin(2.0 * R))**2
    f = A * np.exp(-0.5 * (R**2 - tilt*X1*X2 + tilt*X3*X4 + 0.2*X1*X5 + 2.0*X3*X5))
    if cut is not None:
        f[R > cut] = 0.0
    f = ndimage.gaussian_filter(f, sigma=sigma)
    f = f / np.sum(f)
    return f

f = func(*COORDS, tilt=tilt, cut=xmax)

Define the interpolation grid.

In [ ]:
new_shape = tuple(np.multiply(shape, [1.5, 1.0, 1.5, 1.0, 1.1]).astype(int))
new_coords = [np.linspace(COORDS[i].min(), COORDS[i].max(), new_shape[i]) 
              for i in range(5)]

Step 1: Interpolate over $x_5$ for each $\left\{x_2, x_3, x_4, x_5\right\}$ to get $\tilde{x}_5$.

In [ ]:
_f = np.copy(f)
f_new = np.zeros((shape[0], shape[1], shape[2], shape[3], new_shape[4]))
new_points = new_coords[4]
for i in trange(shape[0]):
    for j in range(shape[1]):
        for k in range(shape[2]):
            for l in range(shape[3]):
                points = COORDS[4][i, j, k, l, :]
                values = _f[i, j, k, l, :]
                f_new[i, j, k, l, :] = interpolate.griddata(
                    points, 
                    values, 
                    new_points, 
                    fill_value=0.0, 
                    method='linear',
                )

Make new coordinate arrays.

In [ ]:
X1 = copy_into_new_dim(X1[:, :, :, :, 0], (new_shape[4],), axis=-1)
X2 = copy_into_new_dim(X2[:, :, :, :, 0], (new_shape[4],), axis=-1)
X3 = copy_into_new_dim(X3[:, :, :, :, 0], (new_shape[4],), axis=-1)
X4 = copy_into_new_dim(X4[:, :, :, :, 0], (new_shape[4],), axis=-1)
X5 = copy_into_new_dim(new_coords[4], (shape[0], shape[1], shape[2], shape[3]), axis=0)
COORDS = [X1, X2, X3, X4, X5]

print('New coordinate arrays:')
for i in range(ndim):
    print(f'X{i + 1}.shape = : {COORDS[i].shape}')

Step 2: Interpolate $x_1$-$x_2$ for each $\left\{x_3, x_4, \tilde{x}_5\right\}$ to get $\tilde{x}_1$ and $\tilde{x}_2$.

In [ ]:
_f = f_new.copy()
f_new = np.zeros((new_shape[0], new_shape[1], shape[2], shape[3], new_shape[4]))
new_points = tuple([C.ravel() for C in np.meshgrid(new_coords[0], new_coords[1], indexing='ij')])
for k in trange(f.shape[2]):
    for l in range(f.shape[3]):   
        for m in range(f.shape[4]):
            points = (
                COORDS[0][:, :, k, l, m].ravel(),
                COORDS[1][:, :, k, l, m].ravel(),
            )
            values = _f[:, :, k, l, m].ravel()
            new_values = interpolate.griddata(
                points,
                values,
                new_points,
                fill_value=0.0,
                method='linear',
            )
            f_new[:, :, k, l, m] = new_values.reshape((new_shape[0], new_shape[1]))

Make new coordinate arrays.

In [ ]:
_X1, _X2 = np.meshgrid(new_coords[0], new_coords[1], indexing='ij')
X1 = copy_into_new_dim(_X1, (shape[2], shape[3], new_shape[4]), axis=-1)
X2 = copy_into_new_dim(_X2, (shape[2], shape[3], new_shape[4]), axis=-1)
X3 = copy_into_new_dim(X3[0, 0, :, :, :], (new_shape[0], new_shape[1]), axis=0)
X4 = copy_into_new_dim(X4[0, 0, :, :, :], (new_shape[0], new_shape[1]), axis=0)
X5 = copy_into_new_dim(X5[0, 0, :, :, :], (new_shape[0], new_shape[1]), axis=0)
COORDS = [X1, X2, X3, X4, X5]

print('New coordinate arrays:')
for i in range(ndim):
    print(f'X{i + 1}.shape = : {COORDS[i].shape}')

Step 3: Interpolate $x_3$-$x_4$ for each $\left\{\tilde{x}_1, \tilde{x}_2, \tilde{x}_5\right\}$ to get $\tilde{x}_3$ and $\tilde{x}_4$.

In [ ]:
_f = f_new.copy()
f_new = np.zeros(new_shape)
new_points = tuple([G.ravel() for G in np.meshgrid(new_coords[2], new_coords[3], indexing='ij')])
for i in trange(new_shape[0]):
    for j in range(new_shape[1]):   
        for m in range(new_shape[4]):
            points = (
                COORDS[2][i, j, :, :, m].ravel(),
                COORDS[3][i, j, :, :, m].ravel(),
            )
            values = _f[i, j, :, :, m].ravel()
            new_values = interpolate.griddata(
                points,
                values,
                new_points,
                fill_value=0.0,
                method='linear',
            )
            f_new[i, j, :, :, m] = new_values.reshape((new_shape[2], new_shape[3]))

Now we compare with the true distribution. 5D distributions are more difficult to compare than 3D distributions. A starting point is to view the full 2D projections.

In [ ]:
f_new = normalize(f_new)
f_true = func(*np.meshgrid(*new_coords, indexing='ij'), cut=xmax, tilt=tilt)

for arr, title in zip([f_true, f_new], ['True', 'Interpolated (1D + 2D + 2D)']):
    axes = utils.corner(arr, coords=new_coords, diag_kind=None, labels=labels, 
                        prof=True, prof_kws=prof_kws, fig_kws=dict(figwidth=5.0))
    axes.format(suptitle=title)
    plt.show()
plt.show()

Looks good to me. There are many other ways to slice the data; here is one:

In [ ]:
ncols = 7
i = new_shape[0] // 2 - 5
j = new_shape[1] // 2 + 5
kk = np.linspace(0, new_shape[4] - 1, ncols + 2)[1:-1].astype(int)
prof_kws = dict(lw=0.5, alpha=0.5, scale=0.1)

fig, axes = pplt.subplots(ncols=ncols, nrows=2, figwidth=7.0, space=0.5)
for row, array in enumerate([f_true, f_new]):
    vmin = np.min(array[i, j, ..., kk])
    vmax = np.max(array[i, j, ..., kk])
    for ax, k in zip(axes[row, :], kk):
        idx = (i, j, slice(None), slice(None), k)
        utils.plot_image(array[idx], x=new_coords[2], y=new_coords[3], ax=ax,
                         profx=True, profy=True, prof_kws=prof_kws,
                         vmin=vmin, vmax=vmax)
        if row == 0:
            ax.set_title(f'{labels[4]} = {new_coords[4][k]:.2f}', fontsize=8.5)
axes.format(
    xticks=[], yticks=[], 
    xlabel=labels[2], ylabel=labels[3],
    leftlabels=['True', 'interp.'],
    leftlabels_kw=dict(fontsize='medium'),
    suptitle=f'Slice: ({labels[0]}, {labels[1]}) = ({new_coords[0][i]:.2f}, {new_coords[1][j]:.2f})'
)